# MineSweeper With A MineSweeperSolver v1.0(?)

- 原本我覺得寫一個遊戲很酷，所以就嘗試寫了一個簡單、陽春的踩地雷。因為某些緣故，它的上和下接起來了，左和右也連在一起！
- 後來我覺得寫一個玩遊戲的程式更酷，所以就嘗試解這個循環版的踩地雷！
- 或許未來會有些學習的方法能應用上，計算地雷的機率、或是找尋特定pattern的解法。

## import packages

In [1]:
import random
from datetime import datetime

## enum

In [2]:
class state:
    ZERO = 0
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    MINE = -1
    FLAG = -2
    COVERED = 9

## class - MineSweeperCircular(Main Game)

<p style="font-size:2pt">主要設計給程式玩，所以暫時沒有UI</p>

In [3]:
class MineSweeperCircular:
    def __init__(self, difficulty = 'beginner', start_position = None):
        self.difficulty = difficulty
        self.initialize(start_position = start_position)
    def initialize(self, difficulty = None, start_position = None):
        random.seed(datetime.now())
        
        if difficulty != None:
            self.difficulty = difficulty
        if self.difficulty == 'beginner':
            self.size = (8, 8)
            self.num_mine = 10
        elif self.difficulty == 'intermediate':
            self.size = (16, 16)
            self.num_mine = 40
        elif self.difficulty == 'expert':
            self.size = (24, 24)
            self.num_mine = 99
        
        self.real_board = [[0]*self.size[1] for i in range(self.size[0])]
        self.mask_board = [[state.COVERED]*self.size[1] for i in range(self.size[0])]
        self.num_flag = 0
        self.num_covered = self.size[0] * self.size[1]
        self.lose = False
        
        # set mines
        if start_position == None:
            for i in random.sample(range(self.size[0] * self.size[1]), self.num_mine):
                self.real_board[i // self.size[1]][i % self.size[1]] = state.MINE
        else:
            num_start_position = start_position[0] * self.size[1] + start_position[1] 
            for i in random.sample(range(self.size[0] * self.size[1] - 1), self.num_mine):
                self.real_board[(i if i < num_start_position else i + 1) // self.size[0]][(i if i < num_start_position else i + 1) % self.size[1]] = state.MINE
        # set numbers
        for i in range(self.size[0]):
            for j in range(self.size[1]):
                if self.real_board[i][j] != state.MINE:
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            self.real_board[i][j] += (self.real_board[(i + di) % self.size[0]][(j + dj) % self.size[1]] == state.MINE)
        # start_position
        if start_position != None:
            self.uncover(start_position)
                    
                
        
    def uncover(self, position):
        position = (position[0] % self.size[0], position[1] % self.size[1])
        if self.mask_board[position[0]][position[1]] != 9:
            return    # do nothing
        if self.real_board[position[0]][position[1]] == state.MINE:
            self.mask_board[position[0]][position[1]] = state.MINE
            self.lose = True
            print("BOOM!!!!!")
            print("If you want to restart, type : >>> s.initialize(difficulty = ??)")
            return
        self.num_covered -= 1
        if self.real_board[position[0]][position[1]] == 0:
            self.mask_board[position[0]][position[1]] = 0
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    self.uncover((position[0] + di, position[1] + dj))
        else:
            self.mask_board[position[0]][position[1]] = self.real_board[position[0]][position[1]]
            
        if self.isWin():
            print("You Win!!!!!")
            print("If you want to restart, type : >>> s.initialize(difficulty = ??)")
    def flag(self, position):
        self.mask_board[position[0]][position[1]] = state.FLAG
        self.num_flag += 1
    def isWin(self):
        return self.num_covered == self.num_mine
    def isLose(self):
        return self.lose
    def display(self):
        print(self.__str__())
    def cheat(self):
        s = "CHEAT\n     "
        for j in range(self.size[1]):
            s += " " + (str(j) if j > 9 else " " + str(j))
        s += "\n     "
        for j in range(self.size[1]):
            s += "___" 
        s += "\n\n"
        for i in range(self.size[0]):
            s += (str(i) if i > 9 else " " + str(i)) + "|  "
            for j in range(self.size[1]):
                if self.real_board[i][j] == state.MINE:
                    s += "  *"
                else:
                    s += "  " + str(self.real_board[i][j]) 
            s += "\n"
        print(s + "\n\n")
    def __str__(self):
        s = "MineSweeperCircular\n     "
        for j in range(self.size[1]):
            s += " " + (str(j) if j > 9 else " " + str(j))
        s += "\n     "
        for j in range(self.size[1]):
            s += "___" 
        s += "\n\n"
        for i in range(self.size[0]):
            s += (str(i) if i > 9 else " " + str(i)) + "|  "
            for j in range(self.size[1]):
                if self.mask_board[i][j] == state.COVERED:
                    s += "  ."
                elif self.mask_board[i][j] == state.FLAG:
                    s += "  f"
                elif self.mask_board[i][j] == state.MINE:
                    s += "  *"
                else:
                    s += "  " + str(self.mask_board[i][j]) 
            s += "\n"
        s += "\n\n剩餘地雷：" + str(self.num_mine - self.num_flag) + "\n\n"
        return s
                
    def __repr__(self):
        
        return self.__str__()
    
        

### 展示一下我寫的踩地雷 -- 起始版面及解答

In [4]:
m = MineSweeperCircular(difficulty = 'expert', start_position = (0,0))
m.display()
m.cheat()

MineSweeperCircular
       0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
     ________________________________________________________________________

 0|    0  0  1  2  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  2  1
 1|    0  1  2  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  2  1  0
 2|    1  2  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  2  0  0
 3|    1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  1  0  0
 4|    2  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  2  2  2
 5|    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 6|    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 7|    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 8|    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 9|    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
10|    .  .  .  .  .  .  .  .  .  .  .  .  

## class - MineSweeperSolver

<p style="font-size:2pt">如果遊戲設計可以保證在第一步踩到0（開一小塊），那解開的機會就會提高不少 </p>

In [5]:
class MineSweeperSolver:
    def __init__(self, game):
        self.game = game
        self.remaining_mine  = game.num_mine
        self.uncovered = set()
        self.covered = {(i, j) for i in range(game.size[0]) for j in range(game.size[1])}
        
        
    def solve(self):
        self.uncover((0, 0))
        self.uncover((self.game.size[0]//2, 0))
        self.uncover((0, self.game.size[0]//2))
        self.uncover((self.game.size[0]//2, self.game.size[0]//2))
        if self.game.isLose():
            print("fail to solve Q_____Q")
            return
        
        while not self.game.isWin() and not self.game.isLose():
            continue_while = False
            min_prob = self.remaining_mine / len(self.covered)
            covered_margin = set()
            pos_to_prob = dict()
            for i in self.uncovered:
                covered_neighbors = [((i[0]+di+self.game.size[0]) % self.game.size[0], (i[1]+dj+self.game.size[1]) %self.game.size[1]) for di in [-1, 0, 1] for dj in [-1, 0, 1] if ((i[0]+di+self.game.size[0]) % self.game.size[0], (i[1]+dj+self.game.size[1]) %self.game.size[1]) in self.covered]
                covered_margin = covered_margin | set(covered_neighbors)
                if not covered_neighbors:
                    self.uncovered.remove(i)
                    continue_while = True
                    break
                prob = (self.game.mask_board[i[0]][i[1]] - len([((i[0]+di+self.game.size[0]) % self.game.size[0], (i[1]+dj+self.game.size[1]) %self.game.size[1]) for di in [-1, 0, 1] for dj in [-1, 0, 1] if self.game.mask_board[(i[0]+di+self.game.size[0]) % self.game.size[0]][(i[1]+dj+self.game.size[1]) %self.game.size[1]] == state.FLAG])) / len(covered_neighbors)
                
                if prob > 0.99:
                    for j in covered_neighbors:
                        self.flag(j)
                    self.uncovered.remove(i)
                    continue_while = True
                    break
                elif prob < 0.01:
                    for j in covered_neighbors:
                        self.uncover(j)
                    self.uncovered.remove(i)
                    continue_while = True
                    break
                elif prob <= min_prob:
                    for j in covered_neighbors:
                        pos_to_prob[j] = prob
                else:
                    for j in covered_neighbors:
                        if j in pos_to_prob:
                            pos_to_prob.pop(j, None)
            if continue_while:
                continue
            if pos_to_prob:
                min_prob = min([pos_to_prob[j] for j in pos_to_prob])
                min_pos = [j for j in pos_to_prob if pos_to_prob[j] == min_prob]
                self.uncover(min_pos[0])
            else:
                for i in self.covered - covered_margin:
                    self.uncover(i)
                    break
        if self.game.isWin():
            print("Problem Solved")
        elif self.game.isLose():
            print("fail to solve Q_____Q")
    def uncover(self, position): 
        position = ((position[0] + self.game.size[0])%self.game.size[0], (position[1] + self.game.size[1])%self.game.size[1])
        if position not in self.covered:
            return
        
        self.covered.remove(position)
        self.game.uncover(position)
        if self.game.mask_board[position[0]][position[1]] == 0:
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    self.uncover((position[0] + di, position[1] + dj))
        else:
            self.uncovered.add(position)
        
    def flag(self, position):
        position = ((position[0] + self.game.size[0])%self.game.size[0], (position[1] + self.game.size[1])%self.game.size[1])
        self.game.flag(position)
        self.remaining_mine -= 1
        self.covered.remove(position)

### 開一個新的遊戲，然後餵給Solver

In [6]:
m = MineSweeperCircular(difficulty = 'expert')
s = solver(m)

In [7]:
s.solve()

You Win!!!!!
If you want to restart, type : >>> s.initialize(difficulty = ??)
Problem Solved


### 展示解出來的成果 ☆ * " ` ' * - . , _ , . - * ' ` " * - . , _ ☆（f 為標記地雷的位置）

In [8]:
m

MineSweeperCircular
       0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
     ________________________________________________________________________

 0|    3  f  f  2  0  0  0  1  f  f  4  f  1  1  f  2  2  f  1  0  1  f  2  2
 1|    2  3  3  2  0  0  1  2  4  f  f  4  2  1  1  1  1  1  1  0  1  1  2  f
 2|    2  1  f  1  0  1  2  f  2  4  f  f  1  1  1  1  1  1  1  1  1  1  2  2
 3|    2  1  1  1  0  1  f  2  1  2  f  4  3  3  f  2  1  f  2  3  f  2  2  f
 4|    3  0  0  1  1  2  1  1  0  1  2  f  3  f  f  2  1  3  f  4  f  2  3  f
 5|    3  0  0  1  f  1  0  0  0  1  2  2  3  f  3  1  0  3  f  4  2  2  4  f
 6|    2  0  1  3  3  3  1  1  0  1  f  1  1  1  1  0  0  3  f  3  2  f  4  f
 7|    1  1  2  f  f  3  f  2  0  1  1  2  1  1  0  0  0  2  f  2  2  f  3  1
 8|    1  2  f  3  2  3  f  3  2  3  2  2  f  1  0  0  0  2  2  2  1  1  1  1
 9|    f  2  1  1  0  1  1  2  f  f  f  3  1  1  0  0  0  1  f  1  0  0  0  1
10|    1  2  2  2  2  1  1  1  2  4  f  2  